In [3]:
import requests
from bs4 import BeautifulSoup
import json
import concurrent.futures
import os
import json
import pandas as pd



In [35]:
# Đọc file Excel
list_file = "all_link_list.csv"
df = pd.read_csv(list_file, encoding='utf-8')

print(len(df))

print(df.head(5))

1156068
   stt area  page cntyCd  baseYy reffNoNm  prlstClsfSrno
0    0   SG     1     SG    2022  2022_04              4
1    1   SG     1     SG    2022  2022_02              2
2    2   SG     1     SG    2022  2022_03              3
3    3   SG     1     SG    2022  2022_01              1
4    4   SG     1     SG    2018  2018_20             55


In [ ]:
def detailinforequest(index,cntyCd,baseYy,reffNoNm,prlstClsfSrno):
# URL của API
    url = 'https://unipass.customs.go.kr/clip/prlstclsfsrch/retrievePrlstClsfCaseDtl.do'

    # Headers
    headers = {
        'Accept': 'text/html, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 'WMONID=N_4A3dINH0I; JSESSIONID=0001O2y8S-yyQQ_lga-knblPV6Lb2QN6662ivRFJ_jK2FPRlLOTKNEsnuhZe_QxBEKq-8U03_kah8EgUgb-_Exa4dmevvhJHEKeNuIq9prRxpJht6ugFASENgz3p_AdMPw4_:eul21',
        'DNT': '1',
        'Origin': 'https://unipass.customs.go.kr',
        'Referer': 'https://unipass.customs.go.kr/clip/index.do',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'isAjax': 'true',
        'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"'
    }

    # Dữ liệu trong body của yêu cầu
    payload = {
        'cntyCd':cntyCd,
        'baseYy':baseYy,
        'reffNoNm':reffNoNm,
        'prlstClsfSrno':prlstClsfSrno
    }

    # Gửi yêu cầu POST với headers và dữ liệu body
    response = requests.post(url, headers=headers, data=payload)
    
    # Kiểm tra phản hồi từ API
    if response.status_code == 200:
        return response.text,index
    else:
        print(f"Row {index} Lỗi")


def extract_data_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    org_table = soup.find("table", class_='org')
    result = {} 
    
    #tìm danh sách các kết quả cần lấy
    soup.find_all("p",) # Tìm thẻ <p> chứa từ khóa "상세결과 항목 :"
    p_tag = soup.find('p', text=lambda x: x and "상세결과 항목 :" in x)

    if p_tag:
        # Tách nội dung sau dấu ":"
        content = p_tag.text.split("상세결과 항목 :")[1].strip()
        # Chuyển nội dung thành set
        result_set = set(content.split(', '))
        print(result_set)
    else:
        print("Không tìm thấy thẻ <p> phù hợp.")
        
    org_table = soup.find("table", class_='org')
    result = {} 

    for catalog in result_set:
        print(f"Đang xử lý: {catalog}")

        if catalog == '관련이미지':
            th_element = org_table.find('th', text='관련 이미지')
            if th_element:  # Kiểm tra nếu thẻ <th> tồn tại
                image_elements = th_element.find_next_sibling('td').find_all('img')
                image_links = [img['src'] for img in image_elements]
                result[catalog] = image_links
            else:
                print(f"Không tìm thấy thẻ <th> cho '{catalog}'")

        else:  # Trường hợp xử lý các catalog khác
            th_element = org_table.find('th', text=catalog)
            if th_element:  # Kiểm tra nếu tìm thấy thẻ <th>
                elems_html_info = th_element.find_next_sibling('td')
                for elem in elems_html_info:
                    if elem.name == 'a':  # Đổi thẻ <a> thành văn bản
                        elem.replace_with(elem.text)
                    elif elem.name == 'br':  # Giữ lại thẻ <br>
                        continue
                elem_text = elems_html_info.decode_contents().replace('\r', '').replace('\t', '').replace('\n', '').strip()
                result[catalog] = elem_text
            else:
                print(f"Không tìm thấy thẻ <th> cho '{catalog}'")
    return result


In [45]:
def save_state(states, thread_index):
    with open(state_file_template.format(thread_index), "w") as f:
        json.dump(states, f)

# Hàm để tải trạng thái, bao gồm cả trạng thái hiện tại trong chunk
def load_state(thread_index):
    try:
        with open(state_file_template.format(thread_index), "r") as f:
            states = json.load(f)
            if isinstance(states, dict) and "start_index" in states:
                return states  # Trả về trạng thái nếu hợp lệ
            else:
                return {"start_index": 0}  # Nếu không có start_index, trả về 0
    except FileNotFoundError:
        return {"start_index": 0}  # Nếu không có file trạng thái, bắt đầu từ đầu


# Hàm xử lý từng phần dữ liệu
def process_data_chunk(data_chunk, thread_index, start_index):
    all_detail_output = []
    current_index = start_index  # Bắt đầu từ trạng thái lưu trước đó

    for index, row in data_chunk:
        cntyCd = row[3]
        baseYy= row[4]
        reffNoNm = row[5]
        prlstClsfSrno = row[6]
        html_content = detailinforequest(index,cntyCd,baseYy,reffNoNm,prlstClsfSrno)
        try:
            # Xử lý logic chính
            html_content,index = detailinforequest(index,cntyCd,baseYy,reffNoNm,prlstClsfSrno)
            detail_output = extract_data_from_html(html_content)
            detail_output["index"] = index
            all_detail_output.append(detail_output)

            # Ghi kết quả vào file riêng cho thread
            with open(output_file_template.format(thread_index), "a", encoding='utf-8') as f:
                f.write(json.dumps(detail_output, ensure_ascii=False) + "\n")

            current_index = idx + 1  # Cập nhật chỉ số đã xử lý

        except Exception as e:
            print(f"Error processing {rrdcNo}: {e}")
            # Lưu trạng thái khi có lỗi, bao gồm index hiện tại
            save_state({"start_index": current_index}, thread_index)
            break  # Dừng lại nếu có lỗi để không làm hỏng chunk này

        # Lưu trạng thái sau khi xử lý xong một phần tử
        save_state({"start_index": current_index}, thread_index)

    return all_detail_output, current_index

In [46]:
# Thư mục để lưu trạng thái và kết quả
output_dir = "org"
os.makedirs(output_dir, exist_ok=True)

# File để lưu trạng thái của vòng lặp
state_file_template = os.path.join(output_dir, "loop_state_{}.json")
output_file_template = os.path.join(output_dir, "output_data_{}.txt")
final_output_file = "final_output_data_org.txt"  # File tổng kết quả
# Bắt đầu vòng lặp từ trạng thái đã lưu
chunk_size = 100000
df_chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# Bắt đầu vòng lặp từ trạng thái đã lưu
saved_states = [load_state(i) for i in range(len(df_chunks))]  # Giả sử có 54 threads

# Sử dụng multithreading để xử lý từng chunk
all_results = []
states = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=len(df_chunks)) as executor:
    futures = {
        executor.submit(process_data_chunk, chunk, idx, saved_states[idx]["start_index"]): idx
        for idx, chunk in enumerate(df_chunks)
    }
    for future in concurrent.futures.as_completed(futures):
        thread_index = futures[future]
        try:
            results, current_index = future.result()
            all_results.extend(results)
            states[thread_index] = {"start_index": current_index}  # Lưu trạng thái của thread
        except Exception as e:
            print(f"Thread {thread_index} encountered an error: {e}")
            # Nếu gặp lỗi trong thread, vẫn lưu trạng thái
            save_state(states[thread_index], thread_index)


Thread 5 encountered an error: too many values to unpack (expected 2)


KeyError: 5